In [1]:
#Library imports
import string
import numpy as np

In [2]:
#Path of our text file
data = '../data/alllines.txt'

#### Initial Processing functions on the text !!

In [3]:
#remove quotation,comma etc punctuation marks using the corresponding string library
def cleanText(line):
    return line.translate(str.maketrans('','', string.punctuation))

In [4]:
def makeDict(dic, key, value):
    if key not in dic:
        dic[key] = []
    dic[key].append(value)

In [5]:
def makeProbabilityDict(list_text):
    probability_dict = {}
    list_len = len(list_text)
    for item in list_text:
        probability_dict[item] = probability_dict.get(item, 0) + 1
    for key, value in probability_dict.items():
        #Calculate probability and store
        probability_dict[key] = value / list_len
    return probability_dict

In [6]:
#Variables to hold the initial states and transitions
first_word = {}
second_word = {}
transitions = {}

#### Build and Train the Markov Model !!

Markov model is a model following the Markov property where the next step depends only on the current step and not the past historical steps. The sequence of events following a Markov Model is called a Markov Chain. In this assignment, I've used a 2-nd order Markov Model where the current step depends on previous 2 states. The steps to approach the problem is as follows:

1) Clean Up the Data
2) Tokenize the text corpus
3) Building the previous and current state-pairs
4) Determine the probability distribution

In [7]:
def markov_model():
    for line in open(data):
        
        #Perform Tokenization-----
        token = cleanText(line.rstrip().lower()).split()
        tokenLength = len(token)
        
        #Building State-Pairs-----
        for i in range(tokenLength):
            curr_token = token[i]
            
            #For the first word, need to calculate initial state distribution
            if i == 0:
                first_word[curr_token] = first_word.get(curr_token, 0) + 1
            else:
                prev_token = token[i - 1]
                
                #For the last word, form additional identification token
                if i == tokenLength - 1:
                    makeDict(transitions, (prev_token, curr_token), 'END')
                
                #For the second word, consider it as 1st order Markov Model
                if i == 1:
                    makeDict(second_word, prev_token, curr_token)
                else:
                    prev2prev_token = token[i - 2]
                    makeDict(transitions, (prev2prev_token, prev_token), curr_token)
                    
    # Building Probability Distribution
    first_word_count = sum(first_word.values())
    for key, value in first_word.items():
        first_word[key] = value / first_word_count
        
    for prev, next_list in second_word.items():
        second_word[prev] = makeProbabilityDict(next_list)
        
    for word_pair, next_list in transitions.items():
        transitions[word_pair] = makeProbabilityDict(next_list)
    
    print('Training of the model is done')

In [8]:
#Call the function to train our model with the given text corpus
markov_model()

Training of the model is done


#### Generating new text !!

After the model is trained, we've First word distribution, Second word distribution and the State transition probaility distributios.
To generate a new text, we'll refer to our above mentioned output and sample out from the distributions.

In [9]:
def sample_word(dictionary):
    num_rand = np.random.random()
    cumulative = 0
    for key, value in dictionary.items():
        cumulative += value
        if num_rand < cumulative:
            return key
    assert(False)

In [10]:
#This is to specify how long our newly generated text would be
number_of_sentences = 10

In [11]:
# Function to generate sample text
def generate_new_text():
    for i in range(number_of_sentences):
        sentence = []
        
        # First Word
        word1 = sample_word(first_word)
        sentence.append(word1)
        # Second word
        word2 = sample_word(second_word[word1])
        sentence.append(word2)
        
        # Subsequent words untill END
        while True:
            word3 = sample_word(transitions[(word1, word2)])
            if word3 == 'END':
                break
            sentence.append(word3)
            word1 = word2
            word2 = word3
        print(' '.join(sentence))

In [12]:
#Generating new text from given corpus
generate_new_text()

i am the turned forth be it so
enter paulina with a wicked lie
if thou mayest discern by that small most greatly lived
what remedy
or in earth
majesty bade me tell thee my lady to the ambassadors of england here
and three times thrice sir
feelingly now
not to whisper wolseyhere makes visitation
to lay my head


#### Prediction !!

In Hidden Markov Model, each state has a probability distribution over the possible output tokens (Emission Probability) in addition to the Transition probaility. Hence, a HMM consists of 2 information : 1) State Path and 2) Token Path(Emitted Sequence). The state path is not visible, we need to infer the underlying state path based on observable token path.

In this assignment,the words in the corpus are hidden states, each input word is an observation. We need to predict what is going to be the completed sentence i.e. the next phrase based on the given phrase.

In [24]:
def prediction(lines):
        lines = cleanText(lines.rstrip().lower()).split()
        word1 = lines[0]
        if len(lines) == 1:
            word2 = [max(self.second_word(word1), key=second_word(word1).get)]
            lines.append(word2)
        else:
            word2 = lines[1]
        while True:
            word3 = max(transitions[(word1, word2)], key=transitions[(word1, word2)].get)
            if word3 == 'END':
                break
            lines.append(word3)
            word1 = word2
            word2 = word3
        print(' '.join(lines) + '.')

In [25]:
#It gives the correct result from our actual corpus
prediction("and breathe")

and breathe shortwinded accents of new broils.


The above approach gives the correct result for the phrase "and breathe", Let's try another one and see what comes up.

In [34]:
prediction("enter king")

enter king henry vi queen margaret and ursula.


In [ ]:
#References: https://www.udemy.com/unsupervised-machine-learning-hidden-markov-models-in-python/